## 0. 前準備

Poetry (python の仮想環境)をインストール

```
brew install poetry
```

### 0.1 graphene-studio のフォルダー内で、仮想環境を生成

```
poetry install
```

### 0.2 VSCode で、test.ipynb を開き、2 で生成した python kernel を指定して実行


## 1. 外場なしの配置

外場は与えず、原子間斥力のみ与えて Quench して原子配置を均一化する。


In [13]:
from logging import getLogger

import numpy as np

from graphenestudio.quench import quench_particles
from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force

logger = getLogger()

# 斥力で配置を調整する原子の個数。最終的に配置する炭素の数ではない。
Natom = 192
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 初期配置。セル相対。
r = random_box(Natom)

# ポテンシャルエネルギーは粒子位置の関数。
pot = lambda r, cell: repulsive_potential(r, cell, repul=4, rc=2.0)
# その勾配。
dpot = lambda r, cell: -repulsive_force(r, cell, repul=4, rc=2.0)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

         Current function value: 4681.351339
         Iterations: 4
         Function evaluations: 145
         Gradient evaluations: 134


/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-LdnOtBEG-py3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1658: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



Quench 前とあとを比較。


In [ ]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go
from itertools import starmap

x = r @ cell

# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x[:, 0],
            y=x[:, 1],
            z=x[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

## 2. graphene の本格的生成

外場を加えたエネルギー関数とその傾きの関数を準備する。


In [1]:
# The gyroid surface and its gradient
from logging import getLogger, basicConfig, INFO

import numpy as np

from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force

from graphenestudio.surface import Gyroid, Sphere, Diamond, PSurface

# from graphenestudio.surface.psurface import PSurface


basicConfig(level=INFO)
logger = getLogger()

# 斥力粒子の個数
Natom = 192
# 外場の強度。小さいと、ぎゅうぎゅうづめになった時に面外に粒子がこぼれる可能性がある。
cost = 100000
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 曲面関数
# surf = Gyroid(eccentricity=0.0)
# surf = Sphere(radius=0.45)
surf = Diamond(eccentricity=0.0)
# surf = PSurface(eccentricity=0.0)

# 斥力粒子の座標(セル相対)
r = random_box(Natom)

# 斥力粒子のポテンシャルエネルギー関数
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
# その勾配
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

### 2.1 Quench し、表示する。


In [2]:
from graphenestudio.quench import quench_particles
import plotly.graph_objects as go

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

/Users/matto/Unison/github/graphene-studio/.venv/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1658: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


         Current function value: 9409.074544
         Iterations: 79
         Function evaluations: 276
         Gradient evaluations: 269


### 2.2

Quench した構造に有限温度を与えてすこし原子が動けるようにし、100 ステップの疑似分子動力学法を実施する。これにより、より歪みが少ない形状にたどりつけるはず。

斥力粒子を均質に配置して三角格子をつくったら、三角格子の重心同士を連結して、双対グラフを生成する。これがグラフェンの構造となる。一連の処理は graphenate 関数の内部で行われる。

できた構造を x, y, z 方向に 2 倍して 2x2x2 の構造を作り、さらにセルを z 方向に 2 倍する。

結果を以下のファイルに書きだす。

1. `.gro` 原子の位置
2. `.top` 結合
3. `.yap` 構造の preview


In [3]:
# import importlib

# graphenestudio = importlib.import_module("graphenestudio")

from graphenestudio import (
    draw_yaplot,
    graphenate,
    dump_gro,
    generate_top,
    replicate_x,
    replicate_y,
    replicate_z,
    extend_z,
)


base = "diamond"
with open(f"{base}.yap", "w") as file:
    count = 100
    for x, cell, g in graphenate(
        Natom,
        cell,
        pot,
        dpot,
        dt=0.0005,  # 0.005
        T=0.1,
    ):

        # x, cell, g = replicate_x(x, cell, g)
        # x, cell, g = replicate_y(x, cell, g)
        # x, cell, g = replicate_z(x, cell, g)
        x, cell, g = extend_z(x, cell, g)

        print(count)
        file.write(draw_yaplot(x, cell, g))
        with open(f"{base}_{count}.itp", "w") as ft:
            ft.write(generate_top(x, cell, g, generate_dihed_list=True))
        with open(f"{base}_{count}.gro", "w") as fg:
            dump_gro(x, cell, g, fg)

        count -= 1
        if count == 0:
            break

/Users/matto/Unison/github/graphene-studio/.venv/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1658: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.

INFO:root:[165, 22, 13, 14] 3-6 cycles in the packing
INFO:root:Tempering


         Current function value: 9225.487995
         Iterations: 87
         Function evaluations: 290
         Gradient evaluations: 283


INFO:root:[170, 23, 12, 16] 3-6 cycles in the packing
INFO:root:[179, 23, 11, 17] 3-6 cycles in the packing
INFO:root:[189, 25, 9, 18] 3-6 cycles in the packing
INFO:root:[200, 28, 11, 17] 3-6 cycles in the packing
INFO:root:[204, 27, 13, 18] 3-6 cycles in the packing
INFO:root:[205, 28, 12, 18] 3-6 cycles in the packing
INFO:root:[209, 28, 12, 17] 3-6 cycles in the packing
INFO:root:[208, 28, 12, 16] 3-6 cycles in the packing
INFO:root:[210, 29, 14, 15] 3-6 cycles in the packing
INFO:root:[211, 31, 14, 15] 3-6 cycles in the packing
INFO:root:[211, 31, 15, 16] 3-6 cycles in the packing
INFO:root:[210, 32, 17, 15] 3-6 cycles in the packing
INFO:root:[216, 32, 19, 12] 3-6 cycles in the packing
INFO:root:[218, 31, 19, 12] 3-6 cycles in the packing
INFO:root:[219, 31, 19, 14] 3-6 cycles in the packing
INFO:root:[221, 30, 19, 14] 3-6 cycles in the packing
INFO:root:[227, 28, 23, 12] 3-6 cycles in the packing
INFO:root:[228, 28, 23, 13] 3-6 cycles in the packing
INFO:root:[231, 28, 22, 13] 3

100


TypeError: Frame.__init__() missing 1 required positional argument: 'remark'

In [7]:
# CIFに変換し、あとはVESTAにまかせるか。

import MDAnalysis as mda
u = mda.Universe('diamond_100.gro') #, 'graphene.top')
ag = u.select_atoms("name C")
ag.write("diamond_100.pdb")

/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-LdnOtBEG-py3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-LdnOtBEG-py3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-LdnOtBEG-py3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-LdnOtBEG-py3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning:

## 2.3 py3Dmol でとりあえず表示


In [2]:
import py3Dmol

# show
view = py3Dmol.view()
view.addModel(open("diamond_100.gro").read(), "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

以下、rindo 研究には使わない。

## 拡張版

曲面を解析的な式ではなく、グリッド上の数値配列として与える。

`cube`や`sphere`は 3 次元グリッドで定義される$w=f(x,y,z)$型の空間関数で、これが$f(x,y,z)=0$を横切る isosurface で、graphene の形状を指定する。


In [13]:
from logging import INFO, basicConfig, getLogger, DEBUG

import numpy as np

from graphenestudio.quench import quench_particles
from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force
from graphenestudio.surface import Grid, Ticks, GridSurfaceFunc


basicConfig(level=INFO)
logger = getLogger()

Natom = 192
cost = 10000
cell = np.diag(np.array([5.0, 5.0, 5.0]))

r = 0.4  # "radius" of the cube, i.e., half length of an edge

XYZ = np.mgrid[-0.5:0.5:11j, -0.5:0.5:11j, -0.5:0.5:11j]
xticks = Ticks(min=-0.5, binw=0.1)
yticks = Ticks(min=-0.5, binw=0.1)
zticks = Ticks(min=-0.5, binw=0.1)

# グリッドタイプの関数定義法
cube = Grid(
    values=np.max(np.abs(XYZ), axis=0) - r, xticks=xticks, yticks=yticks, zticks=zticks
)

sphere = Grid(
    values=np.sum(XYZ**2, axis=0) - r * r,
    xticks=xticks,
    yticks=yticks,
    zticks=zticks,
)

surf = GridSurfaceFunc(sphere)

In [ ]:
r = random_box(Natom)
# x = r @ cell
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

In [ ]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

x2top が生成した top ファイルの中身を確認する。


In [12]:
import networkx as nx

sections = dict()
name = None
with open("gyroid.top") as f:
    for line in f:
        line = line.strip()
        if len(line) > 0 and line[0] == "[":
            name = line
            sections[name] = []
            continue
        if name is not None:
            sections[name].append(line)

# make a graph based on bonds
g0 = nx.Graph()
for line in sections["[ bonds ]"]:
    if len(line) > 0 and line[0] != ";":
        cols = line.split()
        g0.add_edge(int(cols[0]), int(cols[1]))

print(g0.degree)


print(len(sections["[ bonds ]"]))
print(len(sections["[ pairs ]"]))
# split a multiframed gro file into gro files.

import gromacs

with open("gyroid99.gro") as fr:
    for i, frame in enumerate(gromacs.read_gro(fr)):
        break
    g = nx.Graph(
        [
            (i, j)
            for i, j in pl.pairs_iter(
                frame["position"],
                0.18,
                frame["cell"],
                fractional=False,
                distance=False,
            )
        ]
    )

[(1, 2), (370, 3), (388, 3), (2, 2), (3, 3), (400, 3), (4, 3), (5, 3), (16, 3), (132, 3), (6, 3), (9, 2), (7, 3), (17, 3), (8, 3), (13, 3), (27, 3), (173, 3), (392, 3), (10, 2), (11, 3), (12, 3), (52, 2), (60, 2), (166, 3), (167, 3), (14, 3), (15, 3), (20, 3), (32, 3), (18, 3), (38, 3), (19, 3), (26, 3), (22, 3), (25, 3), (21, 3), (24, 3), (135, 3), (131, 3), (23, 3), (144, 3), (147, 3), (140, 3), (28, 3), (31, 3), (29, 3), (39, 3), (30, 3), (67, 3), (201, 3), (193, 3), (34, 3), (138, 3), (33, 3), (35, 3), (139, 3), (143, 3), (41, 3), (36, 3), (37, 3), (312, 3), (42, 3), (321, 3), (40, 3), (43, 3), (66, 3), (342, 3), (44, 3), (45, 2), (319, 3), (332, 3), (390, 3), (46, 2), (53, 3), (55, 3), (47, 3), (50, 2), (51, 3), (54, 3), (48, 3), (49, 2), (93, 3), (331, 3), (389, 3), (56, 3), (94, 3), (59, 1), (84, 3), (57, 3), (58, 3), (99, 3), (83, 3), (104, 3), (160, 3), (61, 2), (62, 3), (182, 1), (161, 3), (186, 3), (63, 3), (64, 3), (65, 3), (73, 3), (398, 3), (78, 3), (177, 3), (176, 3), (6

NameError: name 'pl' is not defined